In [1]:
import os
import numpy as np
import cv2
from skimage.feature import hog
from skimage.color import rgb2gray
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import joblib
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Dataset Path
dataset_path = '/content/drive/MyDrive/M.Tech/Computer Vision/Project/Tests/Dataset_test'

# Define label mapping for multiple classes
label_map = {'cardboard': 0, 'glass': 1, 'metal': 2, 'paper': 3, 'plastic': 4, 'trash': 5}
num_classes = len(label_map)  # Ensure correct number of classes

# Function to extract features (HOG + Color Histogram)
def extract_features(img):
    # Resize Image
    img = cv2.resize(img, (128, 128))

    # Convert to Grayscale
    gray = rgb2gray(img)

    # HOG Feature Extraction
    hog_features, _ = hog(gray, orientations=9, pixels_per_cell=(8, 8),
                          cells_per_block=(2, 2), visualize=True, block_norm='L2-Hys')

    # Color Histogram (RGB)
    hist_r = cv2.calcHist([img], [0], None, [256], [0, 256]).flatten()
    hist_g = cv2.calcHist([img], [1], None, [256], [0, 256]).flatten()
    hist_b = cv2.calcHist([img], [2], None, [256], [0, 256]).flatten()

    # Normalize histograms (avoid division by zero)
    hist_r = hist_r / hist_r.sum() if hist_r.sum() != 0 else hist_r
    hist_g = hist_g / hist_g.sum() if hist_g.sum() != 0 else hist_g
    hist_b = hist_b / hist_b.sum() if hist_b.sum() != 0 else hist_b

    # Concatenate features
    features = np.hstack([hog_features, hist_r, hist_g, hist_b])
    return features

# Load and preprocess images
def load_dataset(dataset_path):
    images = []
    labels = []

    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        if os.path.isdir(folder_path) and folder in label_map:
            for img_name in os.listdir(folder_path):
                img_path = os.path.join(folder_path, img_name)

                # Read image
                img = cv2.imread(img_path)
                if img is None:
                    continue  # Skip corrupted images

                # Extract features
                features = extract_features(img)

                images.append(features)
                labels.append(label_map[folder])  # Multi-class labels

    return np.array(images), np.array(labels)

# Load Dataset
images, labels = load_dataset(dataset_path)

# Normalize Features
scaler = StandardScaler()
images = scaler.fit_transform(images)

# Split Data
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42, stratify=labels)

# Train Classifier (SVM or Random Forest)
classifier = SVC(kernel='linear', C=1.0)  # Change to RandomForestClassifier(n_estimators=100) if needed
classifier.fit(X_train, y_train)

# Evaluate Model
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Model Accuracy:", accuracy)
print(classification_report(y_test, y_pred, target_names=list(label_map.keys())))

# Define save directory and ensure it exists
save_dir = "/content/drive/MyDrive/M.Tech/Computer Vision/Project/Tests2"
os.makedirs(save_dir, exist_ok=True)

# Save Model and Scaler
joblib.dump(classifier, os.path.join(save_dir, "traditional_model.pkl"))
joblib.dump(scaler, os.path.join(save_dir, "scaler.pkl"))

print(f"Model and scaler saved successfully in {save_dir}!")


Mounted at /content/drive
Model Accuracy: 0.6640316205533597
              precision    recall  f1-score   support

   cardboard       0.76      0.78      0.77        81
       glass       0.59      0.65      0.62       100
       metal       0.63      0.61      0.62        82
       paper       0.75      0.74      0.75       119
     plastic       0.56      0.60      0.58        97
       trash       0.92      0.44      0.60        27

    accuracy                           0.66       506
   macro avg       0.70      0.64      0.66       506
weighted avg       0.67      0.66      0.66       506

Model and scaler saved successfully in /content/drive/MyDrive/M.Tech/Computer Vision/Project/Tests2!


In [10]:
import cv2
import numpy as np
import joblib
from skimage.feature import hog
from skimage.color import rgb2gray

# Load the trained model and scaler
model = joblib.load("/content/drive/MyDrive/M.Tech/Computer Vision/Project/Tests2/traditional_model.pkl")
scaler = joblib.load("/content/drive/MyDrive/M.Tech/Computer Vision/Project/Tests2/scaler.pkl")

# Define label mapping (same as training)
label_map = {0: 'cardboard', 1: 'glass', 2: 'metal', 3: 'paper', 4: 'plastic', 5: 'trash'}

# Function to extract features from a single image (HOG + Color Histogram)
def extract_features(img):
    img = cv2.resize(img, (128, 128))  # Resize image

    gray = rgb2gray(img)  # Convert to grayscale

    # Extract HOG features
    hog_features, _ = hog(gray, orientations=9, pixels_per_cell=(8, 8),
                          cells_per_block=(2, 2), visualize=True, block_norm='L2-Hys')

    # Extract Color Histogram
    hist_r = cv2.calcHist([img], [0], None, [256], [0, 256]).flatten()
    hist_g = cv2.calcHist([img], [1], None, [256], [0, 256]).flatten()
    hist_b = cv2.calcHist([img], [2], None, [256], [0, 256]).flatten()

    # Normalize histograms
    hist_r = hist_r / hist_r.sum() if hist_r.sum() != 0 else hist_r
    hist_g = hist_g / hist_g.sum() if hist_g.sum() != 0 else hist_g
    hist_b = hist_b / hist_b.sum() if hist_b.sum() != 0 else hist_b

    # Concatenate features
    features = np.hstack([hog_features, hist_r, hist_g, hist_b])
    return features

# Function to predict image class
def predict_image(image_path):
    img = cv2.imread(image_path)  # Read input image
    if img is None:
        print("Error: Unable to read image!")
        return

    features = extract_features(img)  # Extract features
    features = scaler.transform([features])  # Normalize features using the saved scaler

    prediction = model.predict(features)  # Predict class
    print(f"Predicted Class: {label_map[prediction[0]]}")

# Example usage
image_path = "/content/drive/MyDrive/M.Tech/Computer Vision/Project/Tests/Dataset_test/cardboard/cardboard104.jpg"
predict_image(image_path)


Predicted Class: cardboard
